# Section 1
## Read in database credentials
Below the credentials for connecting to the database are read into variables by extracting the lines from the local file. The local file is not included in the repo.

In [2]:
db_name = ""
db_user = ""
db_pass = ""
db_host = ""
with open("database_credentials.txt") as f:
    db_name = f.readline().strip()
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_host = f.readline().strip()

## Test connection
The next code segment tests to ensure that the database connection is working properly.

In [5]:
import pymysql as pms

In [6]:
try:
    con = pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    print("Successfully connected")
finally:
    if con:
        print("Closing connection")
        con.close()

Successfully connected
Closing connection


## Making a table
The below code creates a sample table.